In [19]:
import glob
import csv
import pandas as pd

path_data = "../ANNOTATION_TRANSFORMED_AIA_ARCHI/DAUDET/*"
liste_annote=[]
for annotateur in glob.glob(path_data):
    for file in glob.glob("%s/*"%annotateur):
        annotation = pd.read_csv(file, on_bad_lines='skip')
        if "Kraken" in file:
            annotation.columns = ["Label"]
            liste_annote.append([annotation])

    
   
   

[[            Label
0             LA;
1       FABRIOUE;
2           \n\n;
3             EE;
4             \n;
...           ...
108278        \n;
108279         =;
108280         =;
108281         =;
108282        \n;

[108283 rows x 1 columns]], [            Label
0             LA;
1       FABRIOUE;
2           \n\n;
3             EE;
4             \n;
...           ...
108278        \n;
108279         =;
108280         =;
108281         =;
108282        \n;

[108283 rows x 1 columns]], [            Label
0             LA;
1       FABRIOUE;
2           \n\n;
3             EE;
4             \n;
...           ...
115971        \n;
115972         =;
115973         =;
115974         =;
115975        \n;

[115976 rows x 1 columns]]]


In [21]:
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.5 MB/s eta 0:00:00m eta 0:00:010:00:01


In [74]:
import pandas as pd
from  nltk.metrics import agreement

# Assume this CSV has the format <tweet_id>,<label>
# anno1_df = pd.read_csv("../ANNOTATION_TRANSFORMED_AIA_ARCHI/DAUDET/CF_DAUDET/CF_DAUDET_petit-chose_Kraken-base_spaCy3.7.2-lg.csv.aia.csv",sep=";", on_bad_lines='skip')
anno1_df = pd.read_csv("../ANNOTATION_TRANSFORMED_AIA_ARCHI/DAUDET/CF_DAUDET/CF_DAUDET_petit-chose_REF_spaCy3.7.2-lg.csv.aia.csv",sep=";", on_bad_lines='skip')
# anno1_df = pd.read_csv("../ANNOTATION_TRANSFORMED_AIA_ARCHI/DAUDET/CF_DAUDET/CF_DAUDET_petit-chose_TesseractFra-PNG_spaCy3.7.2-lg.csv.aia.csv",sep=";", on_bad_lines='skip')
anno1_df.columns = ["Token_1","Label_1"]
# print("pendant",anno1_df)
anno1_df=anno1_df.iloc[:6500]
print("pendant",anno1_df)


# Assume this CSV has the format <tweet_id>,<label>
# anno2_df = pd.read_csv("../ANNOTATION_TRANSFORMED_AIA_ARCHI/DAUDET/LM_DAUDET/LM_DAUDET_petit-chose_Kraken-base_spaCy3.7.2-lg.csv.aia.csv",sep=";",on_bad_lines='skip')
anno2_df = pd.read_csv("../ANNOTATION_TRANSFORMED_AIA_ARCHI/DAUDET/LM_DAUDET/LM_DAUDET_petit-chose_REF_spaCy3.7.2-lg.csv.aia.csv",sep=";",on_bad_lines='skip')
# anno2_df = pd.read_csv("../ANNOTATION_TRANSFORMED_AIA_ARCHI/DAUDET/LM_DAUDET/LM_DAUDET_petit-chose_TesseractFra-PNG_spaCy3.7.2-lg.csv.aia.csv",sep=";",on_bad_lines='skip')

anno2_df.columns = ["Token_2","Label_2"]
anno2_df=anno2_df.iloc[:6500]

# Assume this CSV has the format <tweet_id>,<label>
# anno3_df = pd.read_csv("../ANNOTATION_TRANSFORMED_AIA_ARCHI/DAUDET/MB_DAUDET/MB_DAUDET_petit-chose_Kraken-base_spaCy.3.7.3-lg.csv.aia.csv",sep=";",on_bad_lines='skip')
anno3_df = pd.read_csv("../ANNOTATION_TRANSFORMED_AIA_ARCHI/DAUDET/MB_DAUDET/MB_DAUDET_petit-chose_REF_spaCy3.7.2-lg.csv.aia.csv",sep=";",on_bad_lines='skip')
# anno3_df = pd.read_csv("../ANNOTATION_TRANSFORMED_AIA_ARCHI/DAUDET/MB_DAUDET/MB_DAUDET_petit-chose_TesseractFra-PNG_spaCy3.7.2-lg.csv.aia.csv",sep=";",on_bad_lines='skip')
anno3_df.columns = ["Token_3","Label_3"]
anno3_df=anno3_df.iloc[:6500]

# Join so we have a data frame of labels for each tweet ID
merged_df = anno1_df.join(anno2_df.join(anno3_df))

# We can drop rows where we don't have two labelers if we want
#. I think Krippendorf can account for missing data though, so this
#. step isn't strictly necessary if you only care about Krippendorf
labels_matched_df = merged_df.dropna()
print("labels_matched_df",labels_matched_df)
# Reformat the data into the form AnnotationTask
#  expects.
data = []
for idx, row in labels_matched_df.iterrows():
    data.append(("a1", idx, row["Label_1"]))
    data.append(("a2", idx, row["Label_2"]))
    data.append(("a3", idx, row["Label_3"]))
print("-----------------------------------------------------",data)
    
atask = agreement.AnnotationTask(data=data)

print("Cohen's Kappa:", atask.kappa())
print("Fleiss's Kappa:", atask.multi_kappa())

# This function maps labels into a numeric space,
#. so we can rely on the ordering of labels.
def priority_distance(left_label, right_label):
    mapped_labels = {
        "Critical": 4,
        "High": 3,
        "Medium": 2,
        "Low": 1,
    }
    left_i = mapped_labels[left_label]
    right_i = mapped_labels[right_label]
    
    return abs(left_i - right_i)

# Create a new annotation task with the distance function
# atask_2 = agreement.AnnotationTask(data=data, distance=priority_distance)
atask_2 = agreement.AnnotationTask(data=data)
print("Krippendorf's Alpha:", atask_2.alpha())

pendant        Token_1 Label_1
0     PREMIÈRE     NaN
1       PARTIE     NaN
2           \n     NaN
3            I     NaN
4           --     NaN
...        ...     ...
6495       les     NaN
6496    larmes     NaN
6497         ,     NaN
6498        et     NaN
6499        si     NaN

[6500 rows x 2 columns]
labels_matched_df         Token_1 Label_1    Token_2 Label_2    Token_3 Label_3
21    Languedoc     LOC  Languedoc     LOC  Languedoc     LOC
33         Midi     LOC       Midi     LOC       Midi     LOC
59           M.     PER         M.     PER         M.     PER
60     Eyssette     PER   Eyssette     PER   Eyssette     PER
202      maison     LOC     maison     LOC     maison     LOC
...         ...     ...        ...     ...        ...     ...
6419         Oh    MISC         Oh    MISC         Oh    MISC
6420          !    MISC          !    MISC          !    MISC
6466         M.     PER         M.     PER         M.     PER
6467   Eyssette     PER   Eyssette     PER   Eyssette

In [61]:
from sklearn.metrics import cohen_kappa_score

import pandas as pd


# Assume this CSV has the format <tweet_id>,<label>
anno1_df = pd.read_csv("../ANNOTATION_TRANSFORMED_AIA_ARCHI/DAUDET/CF_DAUDET/CF_DAUDET_petit-chose_Kraken-base_spaCy3.7.2-lg.csv.aia.csv",sep=";", on_bad_lines='skip')
anno1_df.columns = ["Token_1","Label_1"]
# print("pendant",anno1_df)
anno1_df=anno1_df.iloc[:6500]
print("pendant",anno1_df)


# Assume this CSV has the format <tweet_id>,<label>
anno2_df = pd.read_csv("../ANNOTATION_TRANSFORMED_AIA_ARCHI/DAUDET/LM_DAUDET/LM_DAUDET_petit-chose_Kraken-base_spaCy3.7.2-lg.csv.aia.csv",sep=";",on_bad_lines='skip')
anno2_df.columns = ["Token_2","Label_2"]
anno2_df=anno2_df.iloc[:6500]

print(cohen_kappa_score(anno1_df, anno2_df, labels=None))

pendant          Token_1 Label_1
0             LA     NaN
1       FABRIOUE     NaN
2           \n\n     NaN
3             EE     NaN
4             \n     NaN
...          ...     ...
6495          la     NaN
6496  sentinelle     NaN
6497      criait     NaN
6498           :     NaN
6499                 NaN

[6500 rows x 2 columns]


TypeError: '<' not supported between instances of 'float' and 'str'